In [15]:
import cv2
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import swing3dify.time_adjustment as ta
from swing3dify.core import compute_camera_parameters, reconstruct_3D
from swing3dify.visualizations import draw_feature_matches, visualize_3d_points, draw_epipolar_lines, plot_3d_points
from calibration_data import KNOWN_CALIBRATION_DATA
from swing3dify.config import CONF_THR

In [16]:
club1 = pd.read_csv('../data/club/data/red_0712_0016/position_data.csv')
club2 = pd.read_csv('../data/club/data/white_0712_0016/position_data.csv')

conf1 = pd.read_csv('../data/club/data/red_0712_0016/confidence_data.csv')
conf2 = pd.read_csv('../data/club/data/white_0712_0016/confidence_data.csv')

In [17]:
new_club1 = club1.copy()
new_club2 = club2.copy()

new_club1.loc[conf1['BOX_conf'] <= CONF_THR, 'HOSEL_x'] = None
new_club1.loc[conf1['BOX_conf'] <= CONF_THR, 'HOSEL_y'] = None
new_club1.loc[conf1['BOX_conf'] <= CONF_THR, 'GRIP_x'] = None
new_club1.loc[conf1['BOX_conf'] <= CONF_THR, 'GRIP_y'] = None
new_club1.loc[conf1['BOX_conf'] <= CONF_THR, 'TOE_x'] = None
new_club1.loc[conf1['BOX_conf'] <= CONF_THR, 'TOE_y'] = None

new_club2.loc[conf2['BOX_conf'] <= CONF_THR, 'HOSEL_x'] = None
new_club2.loc[conf2['BOX_conf'] <= CONF_THR, 'HOSEL_y'] = None
new_club2.loc[conf2['BOX_conf'] <= CONF_THR, 'GRIP_x'] = None
new_club2.loc[conf2['BOX_conf'] <= CONF_THR, 'GRIP_y'] = None
new_club2.loc[conf2['BOX_conf'] <= CONF_THR, 'TOE_x'] = None
new_club2.loc[conf2['BOX_conf'] <= CONF_THR, 'TOE_y'] = None

In [18]:
club1.interpolate(method='linear', both=True, inplace=True)
club2.interpolate(method='linear', both=True, inplace=True)

In [19]:
delay_frame = ta.calculate_delay_frame(club1, club2, 'HOSEL')
print(f'delay_frame: {delay_frame}')

delay_frame: -32


In [20]:
new_club1['new_frame'] = new_club1['frame']
new_club2['new_frame'] = new_club2['frame'] + delay_frame

In [21]:
index1 = set(new_club1[~new_club1.isnull().any(axis=1)]['new_frame'].values)
index2 = set(new_club2[~new_club2.isnull().any(axis=1)]['new_frame'].values)

common_index = np.array(sorted(list(index1 & index2)))
print(f'common index size: {len(common_index)}')

common index size: 173


In [22]:
new_club1 = new_club1[new_club1['new_frame'].isin(common_index)].copy()
new_club2 = new_club2[new_club2['new_frame'].isin(common_index)].copy()

del new_club1['new_frame']
del new_club2['new_frame']

In [23]:
height, width = 1920, 1080

for c in new_club1.columns[1::2]:
    new_club1[c] = new_club1[c] * width
    new_club2[c] = new_club2[c] * width
for c in new_club1.columns[2::2]:
    new_club1[c] = new_club1[c] * height
    new_club2[c] = new_club2[c] * height

In [24]:
new_club1

,frame,BOX_x,BOX_y,BOX_width,BOX_height,TOE_x,TOE_y,HOSEL_x,HOSEL_y,GRIP_x,GRIP_y
197,198.0,552.000031,1244.500008,68.000001,233.000007,521.127570,1363.391991,529.111948,1348.736458,584.016724,1127.654686
198,199.0,548.000021,1243.000031,74.000001,234.000006,515.492195,1360.772552,523.793836,1347.038040,583.282807,1126.067390
199,200.0,544.000010,1243.500023,81.999998,233.000007,507.839595,1360.188675,516.215458,1345.842018,582.392013,1125.393333
200,201.0,541.500020,1241.500053,84.999998,228.999996,503.410506,1355.591698,511.904172,1341.042709,582.106969,1124.763908
201,202.0,536.500007,1240.499954,90.999997,226.999998,490.959445,1352.799454,501.135242,1345.245895,580.103359,1123.807182
...,...,...,...,...,...,...,...,...,...,...,...
626,627.0,822.000031,1169.000015,254.000006,167.999997,944.581640,1242.026596,915.070925,1251.614113,692.754271,1084.735336
627,628.0,822.000031,1168.500023,254.000006,166.999998,944.099100,1241.979332,915.161626,1250.582314,692.689898,1084.738083
628,629.0,821.499982,1169.000015,254.999993,167.999997,943.989279,1242.361679,914.874394,1251.011467,692.299862,1084.507256
629,630.0,821.499982,1168.500023,254.999993,168.999996,943.837552,1242.341080,914.689000,1251.289330,692.263234,1084.458389


In [25]:
new_club2

,frame,BOX_x,BOX_y,BOX_width,BOX_height,TOE_x,TOE_y,HOSEL_x,HOSEL_y,GRIP_x,GRIP_y
229,230.0,607.500000,1371.000023,137.000005,169.999995,676.444659,1451.800804,653.729095,1453.674545,538.251050,1287.196770
230,231.0,605.999980,1371.000023,133.999997,169.999995,674.165275,1452.307091,651.711388,1453.778000,537.591065,1286.714859
231,232.0,605.000010,1370.500031,132.000000,168.999996,671.971314,1452.453575,649.651129,1453.794708,536.893262,1286.367760
232,233.0,601.999969,1370.000038,127.999998,169.999995,667.865345,1452.777214,645.776324,1453.894272,536.096066,1285.677109
233,234.0,600.500014,1368.499947,124.999998,166.999998,664.710617,1448.862762,641.945164,1451.508293,535.624502,1285.770378
...,...,...,...,...,...,...,...,...,...,...,...
658,659.0,635.500009,1300.499954,167.000003,109.000003,718.624628,1340.266800,696.688621,1361.167946,550.420640,1247.069092
659,660.0,635.500009,1300.499954,167.000003,109.000003,718.993034,1339.920273,696.994843,1361.167488,550.488746,1247.172203
660,661.0,636.499979,1300.499954,167.000003,109.000003,719.976139,1339.766579,697.923746,1361.166458,550.647490,1247.456589
661,662.0,636.499979,1300.499954,167.000003,109.000003,719.987404,1339.755707,697.913575,1361.158905,550.635195,1247.435074
